# Data Platform Team Co-op Case Study

#### Submission Made by Jay Cho

## Instruction

The data below is a stringified table where delimiter is ';' and line_terminator is '\n'.

In [1]:
# Define Data
data =  'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'
print(data)

Airline Code;DelayTimes;FlightCodes;To_From
Air Canada (!);[21, 40];20015.0;WAterLoo_NEWYork
<Air France> (12);[];;Montreal_TORONTO
(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa
12. Air France;[78, 66];;Ottawa_VANcouvER
""".\.Lufthansa.\.""";[12, 33];20055.0;london_MONTreal



## Requirements


Please create a new table with the following transformations on the above command separated data:



Hint: A few libraries are available to help you with this, pandas, duckdb, pyspark, dask, etc. SQL may be able to help as well. You also don’t need to use any of these. Creative solutions are always welcome.



1. FlightCodes column: Some values are null. Flight Codes are supposed to increase by 10 with each row so 1010 and 1030 will have 1020 in the middle. Fill in these missing numbers and make the column an integer column (instead of a float column).



2. To_From column: Should be split into two separate columns for better analysis! Split on '_' to create two new columns respectively. Also, the case of the column is not very readable, convert the column into capital case.



3. Airline Code column: Clean the Airline Codes to have no punctuation except spaces in the middle. E.g. '(Porter Airways. )' should become 'Porter Airways'.

Let's first display the table, how it looks like:

In [2]:
import pandas as pd
from io import StringIO

df = pd.read_csv(StringIO(data), delimiter=';', lineterminator = '\n')

df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


## Defining Issue in the Table

### Requirement 1

We can find that row number 1, and 3 has FlightCodes of NaN. Since Flight Codes are supposed to increase by 10 in integer column:
1. Row number 1's Flight Codes should be 20025
2. Row number 3's Flight Codes should be 20045
3. All floats in column "FlightCodes" should be changed to "int"

In [3]:
# 1. Filling the missing Flight Codes using interpolation:

df['FlightCodes'].interpolate(method='linear', limit_direction='forward', inplace=True)

df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],20025.0,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045.0,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


In [4]:
# 2. Converting the column to int:

df['FlightCodes'] = df['FlightCodes'].astype(int)

df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055,london_MONTreal


### Requirement 2

We want the Departures and Arrivals to be in Separate Column with all capitalized:

1. Separate To_From with delimiter as '_'
2. Remove existing column "To_From"
3. Capitalize all the Departures and Arrivals Name

In [5]:
# 1. Separate To_From with delimiter as '_'
df[['To','From']] = df['To_From'].str.split('_', expand =True)

df

,Airline Code,DelayTimes,FlightCodes,To_From,To,From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork,WAterLoo,NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO,Montreal,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa,CALgary,Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER,Ottawa,VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055,london_MONTreal,london,MONTreal


In [6]:
# 2. Remove existing column "To_From"
df.drop(columns=['To_From'], inplace=True)

df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WAterLoo,NEWYork
1,<Air France> (12),[],20025,Montreal,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary,Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa,VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055,london,MONTreal


In [7]:
# 3. Capitalize all the Departures and Arrivals Name

df['To'] = df['To'].str.upper()
df['From'] = df['From'].str.upper()

df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WATERLOO,NEWYORK
1,<Air France> (12),[],20025,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,""".\.Lufthansa.\.""","[12, 33]",20055,LONDON,MONTREAL


### Requirement 3

We now need to beautify the Airline Code to remove all except for names

1. Remove all punctuation except for spaces
2. Remove digits
3. Remove leading/trailing spaces

In [8]:
# 1. Remove all punctuation except for spaces

df['Airline Code'] = df['Airline Code'].str.replace('[^\w\s]', '', regex=True)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France 12,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12 Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


In [9]:
# 2. Remove digits

df['Airline Code'] = df['Airline Code'].str.replace('\d+', '', regex=True)
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


In [10]:
# 3. Remove leading/trailing spaces
df['Airline Code'] = df['Airline Code'].str.strip()
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


# Result

In [11]:
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


# Improvements

These codes are written specifically to show step my step solution. However, in terms of efficiency, we can improve this code (especially in requirement 3) using:
1. **lambdas** 
2. **re** library

In [14]:
import pandas as pd
from io import StringIO
import re

data =  'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'
df = pd.read_csv(StringIO(data), delimiter=';', lineterminator = '\n')

# Requirement 1
df['FlightCodes'].interpolate(method = 'linear', limit_direction = 'forward', inplace = True)
df['FlightCodes'] = df['FlightCodes'].astype(int)

# Requirement 2
df[['To', 'From']] = df['To_From'].str.upper().str.split('_', expand = True)
df.drop(columns = ['To_From'], inplace = True)

# Requirement 3
df['Airline Code'] = df['Airline Code'].apply(lambda x: re.sub('[^A-Za-z\s]', '', str(x)))
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL
